**Partie II** 

Dans cette partie, nous nous intéressons aux couches de convolution: ce sont les briques de base d'architectures classiques comme le VGG et le ResNet.
Pour comprendre leur effet, nous les manipulons un peu et nous introduisons les opérations de maxpooling qui leur sont souvent associées (**A.**). Puis nous observons comment ont convergé leurs paramètres après apprentissage sur la base de données ImageNet (**B.**). Nous visualisons aussi le signal en sortie des couches de convolution (**C.**).

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os

import torch
import torch.nn as nn   #couches prédéfinies


from torch.utils.data import Dataset, DataLoader

#pour charger des modèles préentraînés sur ImageNet:
import torchvision
from torchvision import datasets, models, transforms

Nous aurons aussi besoin d'accéder au dossier partagé. Pour cela, lancer drive.mount, cliquer sur le lien et entrer le mot de passe.

On vérifie qu'on a bien accès aux données:

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
root = '/content/drive/TP_ENM/data'
#root = '/content/drive/Shareddrives/TP_ENM/data'
os.listdir(root)

**A.** Couches de convolution et maxpooling \\

Avant de définir les couches de convolutions, éxaminons un peu deux modèles standards du deep learning (vgg16 et resnet50). Vous reconnaîtrez, dans les dernières couches, des perceptrons ("classifier"), formés à partir de la classe nn.Linear. \\
Mais, en première partie du réseau, vous voyez apparaître les 'conv2d' qui correspondent à ces fameuses convolutions.



In [4]:
#Deux réseaux de neurones profonds: VGG16 et ResNet50

model = models.vgg16(pretrained=False)
#model = models.resnet50(pretrained=False)
print(model)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

Dans le cas le plus simple, en dimension 1, la "convolution" à un neurone est définie par une relation de la forme: \\
\begin{equation*}
output_i = bias + \sum_{j = 1}^n input_{i + j} \times kernel_{j}  \tag{1}
\end{equation*}
Ici, $kernel$ représente un vecteur de taille $n$ qui contient les paramètres du neurone. Si, par exemple, $kernel$ est positif et de somme 1, c'est une moyenne mobile. Notons enfin que l'opérateur de convolution classique diffère légèrement ($input_{i - j}$ au lieu de  $input_{i + j}$). \\

Le code pytorch est plus compliqué parce que:
- l'input est généralement un batch d'images comprenant plusieurs canaux
- il n'y a pas qu'un neurone

La forme générale est définie [ici](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#torch.nn.Conv2d).

Sur cette page, $C_{in}$, $C_{out}$ et $N$  correspondent respectivement aux nombres de canaux d'entrée, de neurones de la couche, et à la taille du batch. \\
L'opérateur $\star$ cache lui aussi des subtilités: il faut paramétrer la façon dont le noyau se déplace sur l'input (*stride*) et régler la question des bords (*padding*). Ces aspects, sur lesquels nous ne nous attarderons pas, sont plus faciles à comprendre à partir des animations de [Vincent Dumoulin](https://github.com/vdumoulin/conv_arithmetic). \\

Pour illustrer l'effet d'une couche de convolution 2d à un neurone, chargeons une image RGB.

In [ ]:
#%% Une image RGB:
from PIL import Image
path = os.path.join(root, 'cat.jpg')

image = Image.open(path).convert("RGB")
image = image.resize((256,256))
image  

Les opérateurs de "convolution" utilisés par la classe Conv2d sont codés dans torch.nn.functional. Dans les lignes qui suivent, on définit un noyau gaussien, qu'on peut ensuite appliquer à l'image:

In [ ]:
import torch.nn.functional as F  

#définition d'un noyau gaussien de taille 10
x_range = torch.arange(-5, 5, 0.5)
y_range = torch.arange(-5, 5, 0.5)
xx, yy = torch.meshgrid(x_range, y_range)
var = 10

kernel =  (1./(2.*3.14*var)) * torch.exp( - (xx**2 + yy**2)/(2*var) )  #gaussienne

plt.imshow(kernel, cmap='gray')

In [ ]:
#On applique le même noyau à tous les canaux:
kernel = kernel.repeat(1,3,1,1)                        #on repète pour chaque canal R,G,B

#on a ajouté au début la dimension associée à l'indexation de l'image dans le batch ("batch dimension"):
print(kernel.shape)

In [ ]:
#conversion de l'image cat.jpg en torch.tensor:
img = transforms.ToTensor()(image)
img = img.unsqueeze(dim=0)  #ajout de la "batch dimension"

#convolution:
output = F.conv2d(img, kernel)
fig = plt.figure()
plt.imshow(output[0,0,:,:], cmap='gray')

En pratique, on utilise rarement des noyaux de taille aussi importante. Par exemple, dans un VGG, les noyaux sont de taille 3*3. Cela suffit à extraire 
des caractéristiques intéressantes, comme les contours. 

**Exercice:** Appliquer un [filtre de Prewitt](https://fr.wikipedia.org/wiki/Filtre_de_Prewitt) à l'image à partir de la convolution2d pytorch (compléter le code suivant).

In [ ]:
#Gradient horizontal
kernel1 = torch.tensor([[-1.,0.,1.],[-1.,0.,1.],[-1.,0.,1.]])
kernel1 = kernel1.repeat(1,3,1,1)

In [ ]:
#Gradient vertical
kernel2 = 
kernel2 = 

output1 = 
output2 = 

output = (output1**2 + output2**2).sqrt()

fig = plt.figure()
plt.imshow(output[0,0,:,:], cmap='gray')

Le filtre de Prewitt ne peut pourtant pas être encodé dans un réseau standard:
il fait intervenir un carré et une racine. Les non-linéarités qui sont implémentés dans ces réseaux sont plus simples:

- la fonction ReLU. C'est simplement la fonction "partie positive".
- le maxpooling. c'est une forme de sous-échantillonnage. Dans sa forme la plus courante, on divise l'image en carrés de 2*2 pixels, et on renvoie la valeur maximum sur chaque carré. Les dimensions spatiales du tenseur de sortie sont donc divisées par deux.

In [ ]:
#partie positive: fonction reLU
x = torch.rand(1,1,4,4) - 0.5
print(x)
print(x.relu())


#Maxpooling:
x = F.max_pool2d(x, kernel_size = 2)
print(x)

**Exercice:** Quelle est la taille du tenseur en sortie de la couche 30 du vgg16, comparée à celle de l'image d'entrée?

**B.** Les noyaux de convolution après apprentissage

Observons maintenant les couches de convolution après un apprentissage sur un très gros jeu d'images annotées (~1 M) tirées de la base ImageNet ([http://www.image-net.org/challenges/LSVRC/2010/index](https://)). Dans un premier temps, voyons ce que deviennent les noyaux associés aux 64 neurones de la première couche de convolution d'un ResNet50 déjà entraîné:

In [ ]:
#Avant apprentissage:
model = models.resnet50(pretrained=False)
first_layer = model.conv1.weight.data  

print(first_layer.shape)

plt.figure(figsize=(20, 17))
for i in range(first_layer.shape[0]):
    plt.subplot(8, 8, i+1) # 
    plt.imshow(first_layer[i, 0, :, :], cmap='seismic')
    plt.axis('off')
plt.show()

In [ ]:
#Après apprentissage:
model = models.resnet50(pretrained=True)
first_layer = model.conv1.weight.data  

print(first_layer.shape)

plt.figure(figsize=(20, 17))
for i in range(first_layer.shape[0]):
    plt.subplot(8, 8, i+1) # 
    plt.imshow(first_layer[i, 0, :, :], cmap='seismic')
    plt.axis('off')
plt.show()

On retrouve, parmi ces noyaux, des extracteurs de contours similaires au filtre de Prewitt. \\
Les amateurs de traitement du signal reconnaîtront même des patrons très proches des [ondelettes de Morlet](https://www.google.com/search?q=wavelet+morlet+2d&rlz=1C1AVFC_enFR826FR857&hl=fr&sxsrf=ALeKk01sWHdzUO6bRogEv0KFx2gRgOWz_Q:1610077971021&source=lnms&tbm=isch&sa=X&ved=2ahUKEwiIq5fst4vuAhXPxYUKHaskDqAQ_AUoAXoECAUQAw&biw=915&bih=591  ). \\
Ce fait est remarquable: par une simple descente de gradient, on a fait émerger des filtres efficaces pour la reconnaissance des formes.

**Exercice:** Combien la première couche de convolution d'un ResNet50 contient-elle de "neurones" (un noyau par neurone)? \\
Combien y-a-il de poids dans un noyau ?
Combien cette couche contient-elle de poids ? 

**C.** Les cartes de caractéristiques

Voyons maintenant ce que devient l'image à travers le réseau. Reprenons le VGG16, passons-le sur notre image de chat et voyons ce que devient le signal en se propageant dans le réseau. D'abord, voyons si le réseau reconnaît la bonne classe, parmi les mille classes du jeu de données. La liste des classes est [là](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a).


In [ ]:
image = Image.open(path).convert("RGB")
image = image.resize((256,256))

img = transforms.ToTensor()(image)

img = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(img)
img = img.unsqueeze(dim=0)

In [ ]:
model = models.vgg16(pretrained=True)

#sortie brute
output = model(img)                        

#fonction softmax
output = output.softmax(dim=1).detach()    

#prédiction
_, c  =torch.max(output, dim=1)            
print(c)

#"probabilités" associées aux classes
plt.plot(output.squeeze())  

Vous pourrez remarquer qu'en exécutant la dernière cellule plusieurs fois, les probabilités de sortie diffèrent alors que l'entrée ne change pas. Cela tient en fait au "dropout" (refaire un print(model)), qui désactive un sous-ensemble aléatoire des neurones du classifieur (cette opération permet de lutter contre le **surapprentissage**). \\

Pour désactiver le "dropout" et figer le réseau, on passe en mode "eval" avec la commande: \\

*model.eval()*

Maintenant, visualisons le signal en sortie d'une couche de convolution. Pour cela, on peut utiliser la commande [*hook*](https://pytorch.org/tutorials/beginner/former_torchies/nnft_tutorial.html). \\
Les canaux de ces signaux intermédiaires sont appelés **cartes de caractéristiques** (*feature maps*). Voyons les cartes de caractéristiques associées à la première couche de convolution:

In [ ]:
z = []
#fonction qui permet de stocker dans z les cartes de caractéristiques 
def store_layer_output(model, input, output):  
        z.append(output.detach())


model.features[0].register_forward_hook(store_layer_output)
model.features[10].register_forward_hook(store_layer_output)
model.features[17].register_forward_hook(store_layer_output)
model.features[28].register_forward_hook(store_layer_output)

output = model(img)


In [ ]:
for fm in z:
  print(fm.shape)

**Exercice:** Visualiser les cartes de caractéristiques des couches 0, 10, 17, 28. Vous remarquerez qu'au niveau 10, la réponse du neurone est souvent spécifique à un trait précis. 

In [ ]:
feature_maps = z[2]


[...]    

plt.show()
